# How to do Back Testing?

This notebook demonstrate how to perform back testing using `backtest` utility.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2


import matplotlib.pyplot as plt

import a2rl as wi
from a2rl.nbtools import pprint, print  # Enable color outputs when rich is installed.
from a2rl.utils import backtest

## Setup

- Specify 0.8 ratio of data for training. The first 80% of dataframe rows starting from index 0 will be used for training.
- There are 3992 rows for training, 998 rows for test

In [ ]:
wi_df = wi.read_csv_dataset(wi.sample_dataset_path("chiller"))
wi_df.add_value()

# Speed up training for demo purpose
wi_df = wi_df.iloc[:1000]
tokenizer = wi.AutoTokenizer(wi_df, block_size_row=2, train_ratio=0.8)
print(f"Train: {len(tokenizer.train_dataset)}, Test: {len(tokenizer.test_dataset)}")

Train the model. In this example, we are going to train the model using `1 epoch` to speed up, you may need to adjust training configuration for your own use case.

In [ ]:
model_dir = "model-backtest"

config = dict(
    epochs=1,
    batch_size=512,
    embedding_dim=512,
    gpt_n_layer=1,
    gpt_n_head=1,
    learning_rate=6e-4,
    num_workers=0,
    lr_decay=True,
)
config = {"train_config": config}

builder = wi.GPTBuilder(tokenizer, model_dir, config)

In [ ]:
%%time
model = builder.fit()

## Back Test

- Prepare backtest data using a subset of test data. In this case rows with index `-910:-900` which fall within test dataset.
- Let's create a new dataframe assuming it is come hold out set, and then tokenized the dataframe using existing tokenizer.
- Since you have trained the model, you can access your tokenizer from `tokenizer` directly. Alternatively, you can get from `simulator.tokenizer`.

In [ ]:
simulator = wi.Simulator(tokenizer, model)
test_df = wi_df.iloc[-910:-900].reset_index(drop=True)
display(test_df)

test_df_tokenized = tokenizer.field_tokenizer.transform(test_df)
display(test_df_tokenized)

- Let's use the first 2 rows as context, and have `backtest` function predict the next 8 rows.
- `true_df` is a convenient groundtruth dataframe returned to be used for comparison.

In [ ]:
pred_df, true_df = backtest(
    test_df, simulator, start_row=0, context_rows=2, predict_rows=8, return_groudtruth=True
)

The number of rows returned by backtest is `context_rows + predict_rows`.

In [ ]:
pred_df

Now you can compare the states transitoin between simulator and groundtruth based on historical actions.

In [ ]:
fig, axes = plt.subplots(int(len(true_df.states) / 2), 2, figsize=(15, 5))
fig.suptitle("Back Testing for states", fontsize=16)

for idx, col in enumerate(true_df.states):
    true_df[col].plot(ax=axes[idx])
    pred_df[col].plot(ax=axes[idx])
    axes[idx].set_title(col)
    axes[idx].legend(["true", "pred"])